In [ ]:
import wget, json, os
from string import capwords
from pybtex.database import parse_string
import pybtex.errors
from mpcontribs.client import load_client
pybtex.errors.set_strict_mode(False)

In [ ]:
client = load_client('your-api-key-here', host='ml-api.materialsproject.cloud')

In [ ]:
fn = 'dataset_metadata.json'
if not os.path.exists(fn):
    wget.download(f'https://raw.githubusercontent.com/hackingmaterials/matminer/master/matminer/datasets/{fn}')

In [ ]:
data = json.load(open(fn, 'r'))

In [ ]:
for name, info in data.items():
    if not name.startswith('matbench_'):
        continue
    
    columns = {}
    for col, text in info['columns'].items():
        k = col.replace('_', '|').replace('-', '|').replace('(', ' ').replace(')', '')
        columns[k] = text
        
    project = {
        'project': name,
        'is_public': True,
        'owner': 'ardunn@lbl.gov',
        'title': name, # TODO update and set long_title
        'authors': 'A. Dunn, A. Jain',
        'description': info['description'],
        'other': {
            'columns': columns,
            'entries': info['num_entries']
        },
        'urls': {
            'FigShare': info['url']
        }   
    }
    
    print(name)
    for ref in info['bibtex_refs']:
        bib = parse_string(ref, 'bibtex')
        for key, entry in bib.entries.items():
            key_is_doi = key.startswith('doi:')
            url = 'https://doi.org/' + key.split(':', 1)[-1] if key_is_doi else entry.fields.get('url')
            k = 'Zhuo2018' if key_is_doi else capwords(key.replace('_', ''))
            if k.startswith('C2'):
                k = 'Castelli2012'
            elif k.startswith('Landolt'):
                k = 'LB1997'
            elif k == 'Citrine':
                url = 'https://www.citrination.com'
            
            if len(k) > 8:
                k = k[:4] + k[-4:]
            project['urls'][k] = url

    try:
        print(client.projects.create_entry(project=project).result())
    except Exception as ex:
        print(ex)  # TODO should use get_entry to check existence -> use update_entry if project exists